<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 9


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Описание задачи:

Создать базовый класс Order в C#, который будет представлять информацию о
заказах товаров или услуг. На основе этого класса разработать 2-3 производных
класса, демонстрирующих принципы наследования и полиморфизма. В каждом из
классов должны быть реализованы новые атрибуты и методы, а также
переопределены некоторые методы базового класса для демонстрации
полиморфизма.

Требования к базовому классу Order:

1. Атрибуты: ID заказа (OrderId), Дата создания (CreationDate), Сумма заказа
(TotalAmount).
2. Методы:

CalculateTotal(): метод для расчета общей суммы заказа.

AddItem(Item item): метод для добавления элемента в заказ.

RemoveItem(Item item): метод для удаления элемента из заказа.

Требования к производным классам:
1. ОнлайнЗаказ (OnlineOrder): Должен содержать дополнительные атрибуты,
такие как Email клиента (CustomerEmail). Метод AddItem() должен быть
переопределен для добавления информации о способе доставки при
добавлении элемента.
2. ФизическийЗаказ (PhysicalOrder): Должен содержать дополнительные
атрибуты, такие как Адрес доставки (DeliveryAddress).
Метод RemoveItem() должен быть переопределен для добавления
информации о возврате товара при удалении элемента.
3. СпециализированныйЗаказ (SpecializedOrder) (если требуется третий класс):
Должен содержать дополнительные атрибуты, такие как Специальные
условия (SpecialConditions). Метод CalculateTotal() должен быть
переопределен для учета специальных условий при расчете общей суммы
заказа


#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте полиморфизм с перекрытием и прегегрузкой методов, а также generic классы

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
public class Order<T> // обобщенный generic класс
{
    public int OrderId { get; set; }
    public DateTime CreationDate { get; set; }
    public decimal TotalAmount { get; set; }
    protected List<T> Items { get; set; } // позволяет хранить упорядоченные элементы
    public Order(int orderId)
    {
        OrderId = orderId;
        CreationDate = DateTime.Now;
        Items = new List<T>();
        TotalAmount = 0;
    }
    public virtual void CalculateTotal(Func<T, decimal> priceSelector)
    {
        TotalAmount = Items.Sum(priceSelector);
    }
    public virtual void AddItem(T item)
    {
        Items.Add(item);
        CalculateTotal(GetPrice);
    }
    public virtual void AddItem(string itemName, decimal itemPrice) // перегрузка
    {
        var newItem = (dynamic)Activator.CreateInstance(typeof(T), itemName, itemPrice);
        AddItem(newItem);
    }
    public virtual void RemoveItem(T item)
    {
        if (Items.Remove(item))
        {
            CalculateTotal(GetPrice);
        }
    }
    public virtual void RemoveItem(string itemName)
    {
        var itemToRemove = Items.FirstOrDefault(i => ((dynamic)i).Name == itemName);
        if (itemToRemove != null)
        {
            RemoveItem(itemToRemove);
        }
    }
    public virtual void PrintDetails()
    {
        Console.WriteLine($"Заказ ID: {OrderId}, Дата создания: {CreationDate}, Общая сумма: {TotalAmount} р.");
    }
    protected virtual decimal GetPrice(T item)
    {
        return ((dynamic)item).Price; // предполагается, что T имеет свойство Price
    }
}

public class Item // дополнительный класс
{
    public string Name { get; set; }
    public decimal Price { get; set; }
    public Item(string name, decimal price)
    {
        Name = name;
        Price = price;
    }
}

public class OnlineOrder : Order<Item> // обобщенный класс
{
    public string CustomerEmail { get; set; }
    public OnlineOrder(int orderId, string customerEmail) : base(orderId)
    {
        CustomerEmail = customerEmail;
    }
    public override void AddItem(Item item)
    {
        base.AddItem(item);
        PrintAdd(item);
    }
    public override void RemoveItem(Item item)
    {
        base.RemoveItem(item);
        PrintRemove(item);
    }
    private void PrintAdd(Item item)
    {
        Console.WriteLine($"Товар '{item.Name}' добавлен в онлайн-заказ для {CustomerEmail}");
    }
    private void PrintRemove(Item item)
    {
        Console.WriteLine($"Товар '{item.Name}' удален из онлайн-заказа.");
    }
    public override void PrintDetails()
    {
        base.PrintDetails();
        Console.WriteLine($"Email клиента: {CustomerEmail}");
    }
}

public class PhysicalOrder : OnlineOrder // сложное наследование
{
    public string DeliveryAddress { get; set; }
    public PhysicalOrder(int orderId, string deliveryAddress) : base(orderId, "---")
    {
        DeliveryAddress = deliveryAddress;
    }
    public override void PrintDetails()
    {
        base.PrintDetails();
        Console.WriteLine($"Адрес доставки: {DeliveryAddress}");
    }
}

public interface ISpecializedOrder // интерфейс
{
    void SetSpecialConditions(string specialConditions);
    void Sale(Item item);
}

public class SpecializedOrder : Order<Item>, ISpecializedOrder //  множ наследование
{
    public string SpecialConditions { get; private set; }
    public bool HasSpecialConditions { get; private set; } // булева переменная
    public SpecializedOrder(int orderId, string specialConditions) : base(orderId)
    {
        SetSpecialConditions(specialConditions);
    }
    public void SetSpecialConditions(string specialConditions)
    {
        SpecialConditions = specialConditions;
        HasSpecialConditions = !string.IsNullOrEmpty(specialConditions); // проверка на пустоту или null
    }
    public void Sale(Item item)
    {
        Console.WriteLine($"На товар '{item.Name}' действует скидка 10%!");
    }
    public override void CalculateTotal(Func<Item, decimal> priceSelector)
    {
        base.CalculateTotal(priceSelector);
        if (HasSpecialConditions)
        {
            TotalAmount *= 0.9m; // скидка 10%
        }
    }
    public override void PrintDetails()
    {
        base.PrintDetails();
        if (HasSpecialConditions)
        {
            Console.WriteLine($"Специальные условия: {SpecialConditions}");
        }
        else
        {
            Console.WriteLine("Специальные условия отсутствуют.");
        }
    }
}


Item item1 = new Item("Книга", 150);
Item item2 = new Item("Игрушка", item1.Price);
Item item3 = new Item("Подушка", 700);

OnlineOrder onlineOrder = new OnlineOrder(1, "pochta@mail.ru");
onlineOrder.AddItem(item1);
onlineOrder.PrintDetails();
Console.WriteLine();

PhysicalOrder physicalOrder = new PhysicalOrder(2, "ул. Мельникайте, д. 70, г. Тюмень");
physicalOrder.AddItem(item2);
physicalOrder.PrintDetails();
Console.WriteLine();

SpecializedOrder specializedOrder = new SpecializedOrder(3, "Скидка на новый продукт");
specializedOrder.AddItem(item3);
specializedOrder.Sale(item3);
specializedOrder.CalculateTotal(item => item.Price); // передаю функцию для получения цены
specializedOrder.PrintDetails();


Товар 'Книга' добавлен в онлайн-заказ для pochta@mail.ru
Заказ ID: 1, Дата создания: 11/8/2024 3:03:59 PM, Общая сумма: 150 р.
Email клиента: pochta@mail.ru

Товар 'Игрушка' добавлен в онлайн-заказ для ---
Заказ ID: 2, Дата создания: 11/8/2024 3:03:59 PM, Общая сумма: 150 р.
Email клиента: ---
Адрес доставки: ул. Мельникайте, д. 70, г. Тюмень

На товар 'Подушка' действует скидка 10%!
Заказ ID: 3, Дата создания: 11/8/2024 3:03:59 PM, Общая сумма: 630.0 р.
Специальные условия: Скидка на новый продукт
